Workbook for computing threshold given a code and its logical operators.

INPUT: A css code (check matrix and logical operators) + error model

OUTPUT: threshold plot

Want to:
1. Specify a syndrome extraction circuit
2. Apply circuit level noise
3. Decode output

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [2]:
import stim
import pymatching
from csscode.cssCode import cssCode
from codes.example_codes import rsurf_code

In [3]:
rs3 = rsurf_code(3)
css_rs3 = cssCode(rs3[0],rs3[1])

In [4]:
rs5 = rsurf_code(5)
css_rs5 = cssCode(rs5[0],rs5[1])

In [26]:
rs7 = rsurf_code(7)
css_rs7 = cssCode(rs7[0],rs7[1])

In [19]:
def zlogical(L):
    assert L%2==1

    logic_set = set()

    for q in range(L):
        logic_set.add(q)

    return logic_set



In [ ]:
print(zlogical(5))

{0, 1, 2}


In [27]:
for xcheck in css_rs7.check_dict[False]:
    print(len(zlogical(7) & css_rs7.check_dict[False][xcheck])%2)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
syn_extract_circ = stim.Circuit()

Nrounds = 2

for round in range(Nrounds):

    for zcheck in css_rs3.check_dict[True]:
        anc_qub = css_rs3.Nqubits+zcheck
        for dat_qub in css_rs3.check_dict[True][zcheck]:
            syn_extract_circ.append("CX",[dat_qub,anc_qub])
        syn_extract_circ.append("MR",anc_qub)

    for xcheck in css_rs3.check_dict[False]:
        anc_qub = css_rs3.Nqubits+len(css_rs3.check_dict[True])+xcheck

        syn_extract_circ.append("H",anc_qub)
        for dat_qub in css_rs3.check_dict[False][xcheck]:
            syn_extract_circ.append("CX",[anc_qub,dat_qub])
        syn_extract_circ.append("H",anc_qub)
        syn_extract_circ.append("MR",anc_qub)

logic_qub = css_rs3.Nqubits+len(css_rs3.check_dict[False])+len(css_rs5.check_dict[True])

for dat_qub in zlogical(3):
    syn_extract_circ.append("CX", [dat_qub,logic_qub])
    
syn_extract_circ.append("MR",logic_qub)

In [18]:
syn_extract_circ.diagram()

q0: -@---------------------------------------------------------------X-------------------------------------------------------------@-------------------------------------------------------------------------X---------------------------------------------------------------------------
      |                                                               |                                                             |                                                                         |
 q1: -|-@-------------------------------------------------------------|-X-------------X---------------------------------------------|---------@---------------------------------------------------------------|-X--------------X----------------------------------------------------------
      | |                                                             | |             |                                             |         |                                                               | |              |
 q2: -|-|---------------@---------------------------------------------|-|-------------|-X-------------------------------------------|---------|---------------@-----------------------------------------------|-|--------------|-X--------------------------------------------------------
      | |               |                                             | |             | |                                           |         |               |                                               | |              | |
 q3: -|-|-@-------------|-------------@-------------------------------|-|-------------|-|-----------------X-------------------------|---------|-@-------------|-------------@---------------------------------|-|--------------|-|------------------X-------------------------------------
      | | |             |             |                               | |             | |                 |                         |         | |             |             |                                 | |              | |                  |
 q4: -|-|-|-@-----------|-------------|---------------@---------------|-|-------------|-|-X---------------|-X-----------------------|---------|-|-@-----------|-------------|----------------@----------------|-|--------------|-|-X----------------|-X-----------------------------------
      | | | |           |             |               |               | |             | | |               | |                       |         | | |           |             |                |                | |              | | |                | |
 q5: -|-|-|-|-----------|-@-----------|---------------|-@-------------|-|-------------|-|-|-X-------------|-|-----------------------|---------|-|-|-----------|-@-----------|----------------|-@--------------|-|--------------|-|-|-X--------------|-|-----------------------------------
      | | | |           | |           |               | |             | |             | | | |             | |                       |         | | |           | |           |                | |              | |              | | | |              | |
 q6: -|-|-|-|-----------|-|-----------|-@-------------|-|-------------|-|-------------|-|-|-|-------------|-|-X---------------------|---------|-|-|-----------|-|-----------|-@--------------|-|--------------|-|--------------|-|-|-|--------------|-|-X---------------------------------
      | | | |           | |           | |             | |             | |             | | | |             | | |                     |         | | |           | |           | |              | |              | |              | | | |              | | |
 q7: -|-|-|-|-----------|-|-----------|-|-------------|-|-@-----------|-|-------------|-|-|-|-------------|-|-|-X---------------X---|---------|-|-|-----------|-|-----------|-|--------------|-|-@------------|-|--------------|-|-|-|--------------|-|-|-X----------------X--------------
      | | | |           | |           | |             | | |           | |            